In [16]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [17]:
df = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_93596/2010797754.py:1: DtypeWarning: Columns (85,86,88,90,129) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")


In [22]:
df.query("TIPO_ACTIVIDAD == 'HOSPITALIZACIÓN'").groupby("ANIO_EGRESO").size()

ANIO_EGRESO
2019    5434
2020    2850
2021    3086
2022    3217
2023    3594
dtype: int64